## Imports

In [1]:
# General imports 
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import requests as r
from Bio import SeqIO
from io import StringIO
import warnings

warnings.filterwarnings('ignore')

# Import structuremap functions
import structuremap.utils
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score

structuremap.utils.set_logger()

In [2]:
# Set parameters of analysis
analysis_threshold = 20 # number of amino acids either side to analyze

modifications = ["649.3660", "655.3735"] # which modifications we are looking for, as regex strings
heavy_modification = "655.3735" 
light_modification = "649.3660"

## Load Dataset - Labeled Methionines

In [3]:
# Set correct pathing
curr_dir_path_str = "./"
curr_dir_path = os.path.abspath(curr_dir_path_str)

global_data_path_str = "../global_data"
global_data_path = os.path.abspath(global_data_path_str)

print("Current Directory: " + curr_dir_path)
print("Global Data Directory: " + global_data_path)

Current Directory: /Users/ritwiksrinivas/Desktop/Projects/MetML/Mitochondria
Global Data Directory: /Users/ritwiksrinivas/Desktop/Projects/MetML/global_data


In [4]:
# Load initial dataset
data_loc = os.path.join(curr_dir_path, "Mitochondria_DataSet.xlsx")
peptides = pd.read_excel(data_loc, sheet_name="Sheet2")
peptides

Peptide Sequence      Modified Peptide  \
0            AADTIGYPVMIR          AADTIGYPVMIR   
1              IAMQTLDMGR            IAMQTLDMGR   
2             FVGAVDPIMEK           FVGAVDPIMEK   
3    QAQYLGMPINGPFKPDHYRY  QAQYLGMPINGPFKPDHYRY   
4            FADVIPMNLPHR          FADVIPMNLPHR   
..                    ...                   ...   
198     AHMVTLDYTVQVPGTGR     AHMVTLDYTVQVPGTGR   
199            KEQESEVDMK            KEQESEVDMK   
200       RGVMLAVDAVIAELK       RGVMLAVDAVIAELK   
201           MQLLEIITTDK           MQLLEIITTDK   
202              DHFEEAMR              DHFEEAMR   

             Light Modified Peptide          Heavy Modified Peptide  \
0            AADTIGYPVM[649.3660]IR          AADTIGYPVM[655.3735]IR   
1              IAM[649.3660]QTLDMGR            IAM[655.3735]QTLDMGR   
2             FVGAVDPIM[649.3660]EK           FVGAVDPIM[655.3735]EK   
3    QAQYLGM[649.3660]PINGPFKPDHYRY  QAQYLGM[655.3735]PINGPFKPDHYRY   
4            FADVIPM[649.3660]NLPHR          FADVIPM[655.3735]NLPHR   
..                              ...                             ...   
198     AHM[649.3660]VTLDYTVQVPGTGR     AHM[655.3735]VTLDYTVQVPGTGR   
199            KEQESEVDM[649.3660]K            KEQESEVDM[655.3735]K   
200       RGVM[649.3660]LAVDAVIAELK       RGVM[655.3735]LAVDAVIAELK   
201           M[649.3660]QLLEIITTDK           M[655.3735]QLLEIITTDK   
202              DHFEEAM[649.3660]R              DHFEEAM[655.3735]R   

     1 Log2 Ratio HL  10 Log2 Ratio HL  11 Log2 Ratio HL  12 Log2 Ratio HL  \
0                NaN               NaN          3.269016               NaN   
1                NaN          2.783695          3.114945          2.697822   
2                NaN               NaN               NaN               NaN   
3           2.394458          2.380664          2.682897               NaN   
4                NaN               NaN          2.580647               NaN   
..               ...               ...               ...               ...   
198        -1.822067         -1.687501         -1.706146         -1.647231   
199        -1.759694               NaN         -1.821773         -1.960428   
200        -1.812349         -1.980371         -2.307386         -2.492858   
201        -2.286954         -2.700322         -1.941018         -1.860306   
202              NaN         -2.891986         -2.129614               NaN   

     2 Log2 Ratio HL  3 Log2 Ratio HL  ...                Protein  Protein ID  \
0                NaN              NaN  ...   sp|Q8C196|CPSM_MOUSE      Q8C196   
1                NaN              NaN  ...  sp|Q07417|ACADS_MOUSE      Q07417   
2                NaN         2.383482  ...   sp|Q91YI0|ARLY_MOUSE      Q91YI0   
3           2.435014         2.412394  ...   sp|P50247|SAHH_MOUSE      P50247   
4           2.446614         2.450718  ...  sp|P33267|CP2F2_MOUSE      P33267   
..               ...              ...  ...                    ...         ...   
198        -1.793966        -1.407199  ...   sp|Q9QXF8|GNMT_MOUSE      Q9QXF8   
199        -1.781592        -1.988603  ...  sp|Q8K3J1|NDUS8_MOUSE      Q8K3J1   
200        -1.405837        -2.094407  ...   sp|P63038|CH60_MOUSE      P63038   
201        -2.069488              NaN  ...   sp|Q8BMS1|ECHA_MOUSE      Q8BMS1   
202              NaN              NaN  ...   sp|Q01853|TERA_MOUSE      Q01853   

      Entry Name    Gene                                Protein Description  \
0     CPSM_MOUSE    Cps1  Carbamoyl-phosphate synthase [ammonia], mitoch...   
1    ACADS_MOUSE   Acads  Short-chain specific acyl-CoA dehydrogenase, m...   
2     ARLY_MOUSE     Asl                            Argininosuccinate lyase   
3     SAHH_MOUSE    Ahcy                             Adenosylhomocysteinase   
4    CP2F2_MOUSE  Cyp2f2                                Cytochrome P450 2F2   
..           ...     ...                                                ...   
198   GNMT_MOUSE    Gnmt                        Glycine N-methyltransferase   
1

In [5]:
# Canonicalize data - here, drop extra columns
peptides.drop(columns=["Unnamed: 24", "Protein ID.1"], axis=1, inplace=True)
peptides;

In [6]:
# Manual labeling of peptides
label_col_data = ["green"] * 39 + ["white"] * 143 + ["yellow"] * 21
label_col = pd.Series(label_col_data)
peptides["label"] = label_col

#pd.set_option("display.max_rows", None)
#display(peptides)
#pd.reset_option("display.max_rows")
peptides;

In [7]:
unique_uniprotIDs = peptides["Protein ID"].unique()
print("Unique UniProt IDs: \n" + str(unique_uniprotIDs))
print("Number of Unique UniProt IDs: " + str(unique_uniprotIDs.size))

Unique UniProt IDs: 
['Q8C196' 'Q07417' 'Q91YI0' 'P50247' 'P33267' 'P97872' 'Q9DBT9' 'P32020'
 'P50136' 'P26443' 'Q91VS7' 'Q920A5' 'O55125' 'Q05920' 'Q91VR2' 'P51881'
 'P56480' 'Q61425' 'P47962' 'P53395' 'Q921G7' 'Q9CQR4' 'P63017' 'Q62425'
 'P20029' 'O35423' 'Q61102' 'Q9WUR2' 'Q9CQQ7' 'P55096' 'P01942' 'O35490'
 'P38647' 'Q60991' 'Q9Z2I8' 'Q9EQ20' 'Q9Z1J3' 'Q5U458' 'O35129' 'Q99JY0'
 'Q9WTP6' 'Q99LC5' 'P16460' 'Q9CZW5' 'P08226' 'Q03265' 'P63038' 'Q8CAQ8'
 'O35386' 'Q9Z2I0' 'P54869' 'P97742' 'Q9DCN1' 'Q9WUM5' 'P60710' 'P68033'
 'Q9CQ69' 'Q9DBJ1' 'Q9CR62' 'P19783' 'Q8QZY2' 'O70579' 'Q9CQN1' 'Q8BH95'
 'P51660' 'Q8BGY7' 'Q8CC88' 'Q9CRB9' 'Q9Z0X1' 'Q9QXD1' 'Q2TPA8' 'P24270'
 'Q9CPQ8' 'Q80XN0' 'Q925I1' 'Q8CIM7' 'Q9Z1P6' 'Q8VC30' 'Q9R0H0' 'P51658'
 'P16332' 'Q8VDN2' 'Q8BJ64' 'P42125' 'Q99MR8' 'P29758' 'Q5FW57' 'P62821'
 'P52825' 'Q9JKR6' 'Q99KR3' 'Q61733' 'Q61335' 'Q7TNG8' 'Q9DCM2' 'Q8CFA2'
 'Q64433' 'P45952' 'P08249' 'O35459' 'P97450' 'Q9CYR0' 'Q64310' 'Q8K1Z0'
 'Q9CPU0' 'Q80XL6' 'P97807' 'P

In [8]:
# Helper function to get full amino acid sequence for a protein
def get_complete_sequence(cID):
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,"fasta"))

    return str(pSeq[0].seq)

In [9]:
# Intialize and update sequence cache df: get complete AA sequences for relevant proteins
# SLOW - ONLY DO THIS ONCE - CONVERT TO CSV FILE, THEN RE-LOAD FROM THAT FILE

#unknown_sequences_df = pd.DataFrame({"Protein ID": unique_uniprotIDs})
#tqdm.pandas()
#unknown_sequences_df["Complete Sequence"] = unknown_sequences_df["Protein ID"].progress_apply(get_complete_sequence)
#display(unknown_sequences_df)
#
#sequence_cache_df_updated = unknown_sequences_df
#sequence_cache_df_updated.to_csv(os.path.join(global_data_path, "complete_sequence_cache.csv"))
#sequence_cache_df_updated;

In [10]:
# Load cache df: mapping from UniProt IDs to complete AA sequence
path = os.path.join(global_data_path, "complete_sequence_cache.csv")
sequence_cache_df_updated = pd.read_csv(path)
sequence_cache_df_updated.set_index("Unnamed: 0", inplace=True)
sequence_cache_df_updated.index.name = None
sequence_cache_df_updated

Protein ID                                  Complete Sequence
0      Q8C196  MTRILTACKVVKTLKSGFGFANVTTKRQWDFSRPGIRLLSVKAKTA...
1      Q07417  MAAALLARARGPLRRALGVRDWRRLHTVYQSVELPETHQMLRQTCR...
2      Q91YI0  MASESGKLWGGRFVGAVDPIMEKFNSSISYDRHLWNVDVQGSKAYS...
3      P50247  MSDKLPYKVADIGLAAWGRKALDIAENEMPGLMRMREMYSASKPLK...
4      P33267  MDGVSTAILLLLLAVISLSLTFSSRGKGQLPPGPKPLPILGNLLQL...
..        ...                                                ...
28     Q9QX47  MAADIEQVFRSFVVSKFREIQQELSSGRSEGQLNGETNPPIEGNQA...
29     P10605  MWWSLILLSCLLALTSAHDKPSFHPLSDDLINYINKQNTTWQAGRN...
30     P38060  MASVRKAFPRRLVGLTSLRAVSTSSMGTLPKQVKIVEVGPRDGLQN...
31     P11499  MPEEVHHGEEEVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISN...
32     Q02819  MPTSVPRGAPFLLLPPLLMLSAVLAVPVDRAAPPQEDSQATETPDT...

[1779 rows x 2 columns]

In [11]:
peptides_cs = peptides.merge(sequence_cache_df_updated, how="left", on="Protein ID")
peptides_cs # cs means "complete sequence"

Peptide Sequence      Modified Peptide  \
0            AADTIGYPVMIR          AADTIGYPVMIR   
1              IAMQTLDMGR            IAMQTLDMGR   
2             FVGAVDPIMEK           FVGAVDPIMEK   
3    QAQYLGMPINGPFKPDHYRY  QAQYLGMPINGPFKPDHYRY   
4            FADVIPMNLPHR          FADVIPMNLPHR   
..                    ...                   ...   
198     AHMVTLDYTVQVPGTGR     AHMVTLDYTVQVPGTGR   
199            KEQESEVDMK            KEQESEVDMK   
200       RGVMLAVDAVIAELK       RGVMLAVDAVIAELK   
201           MQLLEIITTDK           MQLLEIITTDK   
202              DHFEEAMR              DHFEEAMR   

             Light Modified Peptide          Heavy Modified Peptide  \
0            AADTIGYPVM[649.3660]IR          AADTIGYPVM[655.3735]IR   
1              IAM[649.3660]QTLDMGR            IAM[655.3735]QTLDMGR   
2             FVGAVDPIM[649.3660]EK           FVGAVDPIM[655.3735]EK   
3    QAQYLGM[649.3660]PINGPFKPDHYRY  QAQYLGM[655.3735]PINGPFKPDHYRY   
4            FADVIPM[649.3660]NLPHR          FADVIPM[655.3735]NLPHR   
..                              ...                             ...   
198     AHM[649.3660]VTLDYTVQVPGTGR     AHM[655.3735]VTLDYTVQVPGTGR   
199            KEQESEVDM[649.3660]K            KEQESEVDM[655.3735]K   
200       RGVM[649.3660]LAVDAVIAELK       RGVM[655.3735]LAVDAVIAELK   
201           M[649.3660]QLLEIITTDK           M[655.3735]QLLEIITTDK   
202              DHFEEAM[649.3660]R              DHFEEAM[655.3735]R   

     1 Log2 Ratio HL  10 Log2 Ratio HL  11 Log2 Ratio HL  12 Log2 Ratio HL  \
0                NaN               NaN          3.269016               NaN   
1                NaN          2.783695          3.114945          2.697822   
2                NaN               NaN               NaN               NaN   
3           2.394458          2.380664          2.682897               NaN   
4                NaN               NaN          2.580647               NaN   
..               ...               ...               ...               ...   
198        -1.822067         -1.687501         -1.706146         -1.647231   
199        -1.759694               NaN         -1.821773         -1.960428   
200        -1.812349         -1.980371         -2.307386         -2.492858   
201        -2.286954         -2.700322         -1.941018         -1.860306   
202              NaN         -2.891986         -2.129614               NaN   

     2 Log2 Ratio HL  3 Log2 Ratio HL  ...                Protein  Protein ID  \
0                NaN              NaN  ...   sp|Q8C196|CPSM_MOUSE      Q8C196   
1                NaN              NaN  ...  sp|Q07417|ACADS_MOUSE      Q07417   
2                NaN         2.383482  ...   sp|Q91YI0|ARLY_MOUSE      Q91YI0   
3           2.435014         2.412394  ...   sp|P50247|SAHH_MOUSE      P50247   
4           2.446614         2.450718  ...  sp|P33267|CP2F2_MOUSE      P33267   
..               ...              ...  ...                    ...         ...   
198        -1.793966        -1.407199  ...   sp|Q9QXF8|GNMT_MOUSE      Q9QXF8   
199        -1.781592        -1.988603  ...  sp|Q8K3J1|NDUS8_MOUSE      Q8K3J1   
200        -1.405837        -2.094407  ...   sp|P63038|CH60_MOUSE      P63038   
201        -2.069488              NaN  ...   sp|Q8BMS1|ECHA_MOUSE      Q8BMS1   
202              NaN              NaN  ...   sp|Q01853|TERA_MOUSE      Q01853   

      Entry Name    Gene                                Protein Description  \
0     CPSM_MOUSE    Cps1  Carbamoyl-phosphate synthase [ammonia], mitoch...   
1    ACADS_MOUSE   Acads  Short-chain specific acyl-CoA dehydrogenase, m...   
2     ARLY_MOUSE     Asl                            Argininosuccinate lyase   
3     SAHH_MOUSE    Ahcy                             Adenosylhomocysteinase   
4    CP2F2_MOUSE  Cyp2f2                                Cytochrome P450 2F2   
..           ...     ...                                                ...   
198   GNMT_MOUSE    Gnmt                        Glycine N-methyltransferase   
1

In [12]:
# Create regex pattern to identify desired modifications
def create_modifications_pattern(modifications):

    whole, mantissa = modifications[0].split(".")
    pattern = r"M\[{}\.{}\]".format(whole, mantissa)

    for i in range(1, len(modifications)):
        whole, mantissa = modifications[i].split(".")
        pattern += r"|M\[{}\.{}\]".format(whole, mantissa)
    
    return pattern

modifications_pattern = create_modifications_pattern(modifications)
print(modifications_pattern)

M\[649\.3660\]|M\[655\.3735\]


In [13]:
peptides_cs["Sequence Location"] = pd.Series([a.find(b) for a, b in zip(peptides_cs["Complete Sequence"], peptides_cs["Peptide Sequence"])])
peptides_cs;

In [14]:
peptides_cs["Sequence Length"] = peptides_cs["Peptide Sequence"].str.len()
peptides_cs;

In [15]:
# Sanity check - ensure sequence indexing is correct
temp = [A[B:B+C] for A, B, C in zip(peptides_cs["Complete Sequence"], peptides_cs["Sequence Location"], peptides_cs["Sequence Length"])]
(temp == peptides_cs["Peptide Sequence"]).value_counts()

Peptide Sequence
True    203
Name: count, dtype: int64

In [16]:
# Extract left prefix of modified methionine (for indexing purposes)

IUPACCodes = "ACDEFGHIKLMNPQRSTVWY"
filtering = lambda string: ''.join([char for char in string if char in IUPACCodes])

left_prefix_pattern = "(.*)(" + modifications_pattern + ")"
print(left_prefix_pattern)

peptides_cs["Left Prefix"] = peptides_cs["Light Modified Peptide"].str.extract(left_prefix_pattern)[0]
peptides_cs["Left Prefix"] = peptides_cs["Left Prefix"].map(filtering)
peptides_cs["Left Prefix Length"] = peptides_cs["Left Prefix"].str.len()

peptides_cs;

(.*)(M\[649\.3660\]|M\[655\.3735\])


In [17]:
peptides_cs["Methionine Location"] = peptides_cs["Sequence Location"] + peptides_cs["Left Prefix Length"]
peptides_cs;

In [18]:
# Sanity check - ensure methionine locations are correct
temp = [A[B] for A, B in zip(peptides_cs["Complete Sequence"], peptides_cs["Methionine Location"])]
temp.count("M") == len(temp)

True

In [19]:
# Compute left/right analysis sequences based on threshold
peptides_cs[f"Left {analysis_threshold}"] = [A[B-analysis_threshold:B]  if (B - analysis_threshold >= 0) else A[0:B-1] for A, B in zip(peptides_cs["Complete Sequence"], peptides_cs["Methionine Location"])]
peptides_cs[f"Right {analysis_threshold}"] = [A[B+1:B+1+analysis_threshold] for A, B in zip(peptides_cs["Complete Sequence"], peptides_cs["Methionine Location"])]
peptides_cs

Peptide Sequence      Modified Peptide  \
0            AADTIGYPVMIR          AADTIGYPVMIR   
1              IAMQTLDMGR            IAMQTLDMGR   
2             FVGAVDPIMEK           FVGAVDPIMEK   
3    QAQYLGMPINGPFKPDHYRY  QAQYLGMPINGPFKPDHYRY   
4            FADVIPMNLPHR          FADVIPMNLPHR   
..                    ...                   ...   
198     AHMVTLDYTVQVPGTGR     AHMVTLDYTVQVPGTGR   
199            KEQESEVDMK            KEQESEVDMK   
200       RGVMLAVDAVIAELK       RGVMLAVDAVIAELK   
201           MQLLEIITTDK           MQLLEIITTDK   
202              DHFEEAMR              DHFEEAMR   

             Light Modified Peptide          Heavy Modified Peptide  \
0            AADTIGYPVM[649.3660]IR          AADTIGYPVM[655.3735]IR   
1              IAM[649.3660]QTLDMGR            IAM[655.3735]QTLDMGR   
2             FVGAVDPIM[649.3660]EK           FVGAVDPIM[655.3735]EK   
3    QAQYLGM[649.3660]PINGPFKPDHYRY  QAQYLGM[655.3735]PINGPFKPDHYRY   
4            FADVIPM[649.3660]NLPHR          FADVIPM[655.3735]NLPHR   
..                              ...                             ...   
198     AHM[649.3660]VTLDYTVQVPGTGR     AHM[655.3735]VTLDYTVQVPGTGR   
199            KEQESEVDM[649.3660]K            KEQESEVDM[655.3735]K   
200       RGVM[649.3660]LAVDAVIAELK       RGVM[655.3735]LAVDAVIAELK   
201           M[649.3660]QLLEIITTDK           M[655.3735]QLLEIITTDK   
202              DHFEEAM[649.3660]R              DHFEEAM[655.3735]R   

     1 Log2 Ratio HL  10 Log2 Ratio HL  11 Log2 Ratio HL  12 Log2 Ratio HL  \
0                NaN               NaN          3.269016               NaN   
1                NaN          2.783695          3.114945          2.697822   
2                NaN               NaN               NaN               NaN   
3           2.394458          2.380664          2.682897               NaN   
4                NaN               NaN          2.580647               NaN   
..               ...               ...               ...               ...   
198        -1.822067         -1.687501         -1.706146         -1.647231   
199        -1.759694               NaN         -1.821773         -1.960428   
200        -1.812349         -1.980371         -2.307386         -2.492858   
201        -2.286954         -2.700322         -1.941018         -1.860306   
202              NaN         -2.891986         -2.129614               NaN   

     2 Log2 Ratio HL  3 Log2 Ratio HL  ...  Log2HL avg   label  \
0                NaN              NaN  ...    3.289988   green   
1                NaN              NaN  ...    2.789195   green   
2                NaN         2.383482  ...    2.555706   green   
3           2.435014         2.412394  ...    2.538998   green   
4           2.446614         2.450718  ...    2.389057   green   
..               ...              ...  ...         ...     ...   
198        -1.793966        -1.407199  ...   -1.857135  yellow   
199        -1.781592        -1.988603  ...   -1.871741  yellow   
200        -1.405837        -2.094407  ...   -1.950460  yellow   
201        -2.069488              NaN  ...   -2.270072  yellow   
202              NaN              NaN  ...   -2.507726  yellow   

                                     Complete Sequence  Sequence Location  \
0    MTRILTACKVVKTLKSGFGFANVTTKRQWDFSRPGIRLLSVKAKTA...                575   
1    MAAALLARARGPLRRALGVRDWRRLHTVYQSVELPETHQMLRQTCR...                262   
2    MASESGKLWGGRFVGAVDPIMEKFNSSISYDRHLWNVDVQGSKAYS...                 12   
3    MSDKLPYKVADIGLAAWGRKALDIAENEMPGLMRMREMYSASKPLK...                412   
4    MDGVSTAILLLLLAVISLSLTFSSRGKGQLPPGPKPLPILGNLLQL...                358   
..                                                 ...                ...   
198  MVDSVYRTRSLGVAAEGLPDQYADGEAARVWQLYIGDTRSRTAEYK...                213   
199  MYRLSSSMLPRALAQAMRTGHLNGQSLHSSAVAATYKYVNKKEQES...                 41   
200  MLRLPTVLRQMRPVSRALAPHLTRAYAKDVKFGADARALMLQGVDL...                141   
201  MVASRAIGSLSRFSAFRILRSRGCICRSFTTSSALLTR

# Download Alphafold Data - Labeled Methionines

In [20]:
# Path for alphafold protein data

alphafold_path_str = "../alphafold_data/"
alphafold_path = os.path.abspath(alphafold_path_str)

cif_dir = os.path.join(alphafold_path, "cif")
pae_dir = os.path.join(alphafold_path, "pae")

print(alphafold_path)
print(cif_dir)
print(pae_dir)

/Users/ritwiksrinivas/Desktop/Projects/MetML/alphafold_data
/Users/ritwiksrinivas/Desktop/Projects/MetML/alphafold_data/cif
/Users/ritwiksrinivas/Desktop/Projects/MetML/alphafold_data/pae


In [21]:
# Set uniprot IDs to use
unique_uniprotIDs = peptides_cs["Protein ID"].unique()
print("Unique UniProt IDs: \n" + str(unique_uniprotIDs))
print("Number of Unique UniProt IDs: " + str(unique_uniprotIDs.size))

Unique UniProt IDs: 
['Q8C196' 'Q07417' 'Q91YI0' 'P50247' 'P33267' 'P97872' 'Q9DBT9' 'P32020'
 'P50136' 'P26443' 'Q91VS7' 'Q920A5' 'O55125' 'Q05920' 'Q91VR2' 'P51881'
 'P56480' 'Q61425' 'P47962' 'P53395' 'Q921G7' 'Q9CQR4' 'P63017' 'Q62425'
 'P20029' 'O35423' 'Q61102' 'Q9WUR2' 'Q9CQQ7' 'P55096' 'P01942' 'O35490'
 'P38647' 'Q60991' 'Q9Z2I8' 'Q9EQ20' 'Q9Z1J3' 'Q5U458' 'O35129' 'Q99JY0'
 'Q9WTP6' 'Q99LC5' 'P16460' 'Q9CZW5' 'P08226' 'Q03265' 'P63038' 'Q8CAQ8'
 'O35386' 'Q9Z2I0' 'P54869' 'P97742' 'Q9DCN1' 'Q9WUM5' 'P60710' 'P68033'
 'Q9CQ69' 'Q9DBJ1' 'Q9CR62' 'P19783' 'Q8QZY2' 'O70579' 'Q9CQN1' 'Q8BH95'
 'P51660' 'Q8BGY7' 'Q8CC88' 'Q9CRB9' 'Q9Z0X1' 'Q9QXD1' 'Q2TPA8' 'P24270'
 'Q9CPQ8' 'Q80XN0' 'Q925I1' 'Q8CIM7' 'Q9Z1P6' 'Q8VC30' 'Q9R0H0' 'P51658'
 'P16332' 'Q8VDN2' 'Q8BJ64' 'P42125' 'Q99MR8' 'P29758' 'Q5FW57' 'P62821'
 'P52825' 'Q9JKR6' 'Q99KR3' 'Q61733' 'Q61335' 'Q7TNG8' 'Q9DCM2' 'Q8CFA2'
 'Q64433' 'P45952' 'P08249' 'O35459' 'P97450' 'Q9CYR0' 'Q64310' 'Q8K1Z0'
 'Q9CPU0' 'Q80XL6' 'P97807' 'P

In [22]:
# Download cif data for proteins
# SLOW THE FIRST TIME - caches the relevant cif data
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=unique_uniprotIDs,
    out_folder=cif_dir
)

100%|██████████| 113/113 [00:00<00:00, 35919.39it/s]

2024-10-17 15:34:35> Valid proteins: 0
2024-10-17 15:34:35> Invalid proteins: 0
2024-10-17 15:34:35> Existing proteins: 113


In [23]:
# Download pae data for proteins
# SLOW THE FIRST TIME - caches the relevant pae data
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=unique_uniprotIDs,
    out_folder=pae_dir, 
)

100%|██████████| 113/113 [00:00<00:00, 51623.61it/s]

2024-10-17 15:34:35> Valid proteins: 0
2024-10-17 15:34:35> Invalid proteins: 0
2024-10-17 15:34:35> Existing proteins: 113


## Construct Alphafold Dataframe (Calculate Accessibilities) - Labeled Methionines

In [24]:
# Format alphafold data into dataframe
alphafold_annotation = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=unique_uniprotIDs)
alphafold_annotation

100%|██████████| 1630/1630 [00:08<00:00, 200.45it/s]


protein_id  protein_number AA  position  quality  x_coord_c  x_coord_ca  \
0         O35129               1  M         1    45.38    -35.172     -33.707   
1         O35129               1  A         2    50.78    -38.162     -37.117   
2         O35129               1  Q         3    52.40    -38.996     -38.671   
3         O35129               1  N         4    51.23    -39.258     -38.274   
4         O35129               1  L         5    50.02    -42.031     -40.574   
...          ...             ... ..       ...      ...        ...         ...   
52926     Q9Z2I8             113  S       429    94.38     26.176      27.466   
52927     Q9Z2I8             113  V       430    91.54     24.398      24.087   
52928     Q9Z2I8             113  A       431    73.79     23.429      23.961   
52929     Q9Z2I8             113  K       432    62.38     22.744      23.748   
52930     Q9Z2I8             113  K       433    51.30     20.532      20.426   

       x_coord_cb  x_coord_n  y_coord_c  ...  z_coord_ca  z_coord_cb  \
0         -32.901    -33.031      3.159  ...      51.060      49.845   
1         -37.283    -35.709      2.229  ...      50.385      50.224   
2         -38.089    -37.769      4.596  ...      53.616      55.022   
3         -36.904    -38.080      7.104  ...      52.434      52.182   
4         -40.297    -39.604      6.273  ...      49.354      48.180   
...           ...        ...        ...  ...         ...         ...   
52926      27.350     27.774     14.096  ...     -18.565     -18.413   
52927      23.021     25.313     16.707  ...     -18.909     -17.814   
52928      23.247     23.808     19.412  ...     -21.302     -22.660   
52929      24.902     24.238     22.331  ...     -19.459     -19.112   
52930      19.015     21.457     24.555  ...     -20.722     -20.474   

       z_coord_n  secondary_structure  structure_group  BEND  HELX STRN TURN  \
0         51.772         unstructured     unstructured     0     0    0    0   
1         50.740           TURN_TY1_P             TURN     0     0    0    1   
2         52.617         HELX_RH_AL_P             HELX     0     1    0    0   
3         52.692         HELX_RH_AL_P             HELX     0     1    0    0   
4         50.434         HELX_RH_AL_P             HELX     0     1    0    0   
...          ...                  ...              ...   ...   ...  ...  ...   
52926    -17.274         HELX_RH_AL_P             HELX     0     1    0    0   
52927    -18.401                 BEND             BEND     1     0    0    0   
52928    -20.673         unstructured     unstructured     0     0    0    0   
52929    -20.119         unstructured     unstructured     0     0    0    0   
52930    -20.058         unstructured     unstructured     0     0    0    0   

       unstructured  
0                 1  
1                 0  
2                 0  
3                 0  
4                 0  
...             ...  
52926             0  
52927             0  
52928             1  
52929             1  
52930             1  

[52931 rows x 24 columns]

In [25]:
# Calculate full sphere exposure -> radius = 2
exposure_sphere_rad2 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=2, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad2;

100%|██████████| 113/113 [00:01<00:00, 57.83it/s] 


In [26]:
alphafold_accessibility = alphafold_annotation.merge(
    exposure_sphere_rad2, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [27]:
# Calculate full sphere exposure -> radius = 3
exposure_sphere_rad3 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=3, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad3;

100%|██████████| 113/113 [00:00<00:00, 171.06it/s]


In [28]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad3, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [29]:
# Calculate full sphere exposure -> radius = 4
exposure_sphere_rad4 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=4, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4;

100%|██████████| 113/113 [00:00<00:00, 184.82it/s]


In [30]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad4, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [31]:
# Calculate full sphere exposure -> radius = 4.5
exposure_sphere_rad4_5 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=4.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4_5;

100%|██████████| 113/113 [00:00<00:00, 142.72it/s]


In [32]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad4_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [33]:
# Calculate full sphere exposure -> radius = 5
exposure_sphere_rad5 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5;

100%|██████████| 113/113 [00:00<00:00, 168.79it/s]


In [34]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [35]:
# Calculate full sphere exposure -> radius = 5.5
exposure_sphere_rad5_5 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=5.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5_5;

100%|██████████| 113/113 [00:00<00:00, 163.62it/s]


In [36]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad5_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [37]:
# Calculate full sphere exposure -> radius = 6
exposure_sphere_rad6 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=6, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6;

100%|██████████| 113/113 [00:00<00:00, 176.54it/s]


In [38]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad6, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [39]:
# Calculate full sphere exposure -> radius = 6.5
exposure_sphere_rad6_5 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=6.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6_5;

100%|██████████| 113/113 [00:00<00:00, 170.33it/s]


In [40]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad6_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [41]:
# Calculate full sphere exposure -> radius = 7
exposure_sphere_rad7 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=7, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7;

100%|██████████| 113/113 [00:00<00:00, 158.94it/s]


In [42]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad7, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [43]:
# Calculate full sphere exposure -> radius = 7.5
exposure_sphere_rad7_5 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=7.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7_5;

100%|██████████| 113/113 [00:00<00:00, 143.78it/s]


In [44]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad7_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [45]:
# Calculate full sphere exposure -> radius = 8
exposure_sphere_rad8 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=8, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad8;

100%|██████████| 113/113 [00:00<00:00, 160.58it/s]


In [46]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad8, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [47]:
# Calculate full sphere exposure -> radius = 12
exposure_sphere_rad12 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad12;

100%|██████████| 113/113 [00:00<00:00, 133.62it/s]


In [48]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [49]:
# Calculate full sphere exposure -> radius = 18
exposure_sphere_rad18 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=18, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad18;

100%|██████████| 113/113 [00:01<00:00, 91.74it/s]


In [50]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad18, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [51]:
# Calculate full sphere exposure -> radius = 24
exposure_sphere_rad24 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad24;

100%|██████████| 113/113 [00:01<00:00, 61.65it/s]


In [52]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_sphere_rad24, how='left', on=['protein_id','AA','position'])
alphafold_accessibility;

In [53]:
# Calculate part sphere exposure -> angle = 70, radius = 12
exposure_ang70_rad12 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)
exposure_ang70_rad12;

100%|██████████| 113/113 [00:00<00:00, 131.47it/s]


In [54]:
alphafold_accessibility = alphafold_accessibility.merge(
    exposure_ang70_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility

protein_id  protein_number AA  position  quality  x_coord_c  x_coord_ca  \
0         O35129               1  M         1    45.38    -35.172     -33.707   
1         O35129               1  A         2    50.78    -38.162     -37.117   
2         O35129               1  Q         3    52.40    -38.996     -38.671   
3         O35129               1  N         4    51.23    -39.258     -38.274   
4         O35129               1  L         5    50.02    -42.031     -40.574   
...          ...             ... ..       ...      ...        ...         ...   
52926     Q9Z2I8             113  S       429    94.38     26.176      27.466   
52927     Q9Z2I8             113  V       430    91.54     24.398      24.087   
52928     Q9Z2I8             113  A       431    73.79     23.429      23.961   
52929     Q9Z2I8             113  K       432    62.38     22.744      23.748   
52930     Q9Z2I8             113  K       433    51.30     20.532      20.426   

       x_coord_cb  x_coord_n  y_coord_c  ...  nAA_5.5_180_pae  nAA_6_180_pae  \
0         -32.901    -33.031      3.159  ...                1              1   
1         -37.283    -35.709      2.229  ...                0              2   
2         -38.089    -37.769      4.596  ...                1              2   
3         -36.904    -38.080      7.104  ...                0              2   
4         -40.297    -39.604      6.273  ...                2              2   
...           ...        ...        ...  ...              ...            ...   
52926      27.350     27.774     14.096  ...                2              2   
52927      23.021     25.313     16.707  ...                2              2   
52928      23.247     23.808     19.412  ...                2              2   
52929      24.902     24.238     22.331  ...                2              2   
52930      19.015     21.457     24.555  ...                1              1   

       nAA_6.5_180_pae  nAA_7_180_pae  nAA_7.5_180_pae  nAA_8_180_pae  \
0                    1              1                1              1   
1                    2              2                2              2   
2                    2              2                2              2   
3                    2              2                2              2   
4                    2              2                2              2   
...                ...            ...              ...            ...   
52926                4              4                5              5   
52927                2              5                5              7   
52928                2              2                2              2   
52929                2              2                2              2   
52930                1              1                1              1   

       nAA_12_180_pae nAA_18_180_pae nAA_24_180_pae  nAA_12_70_pae  
0                   3              5              7              0  
1                   4              6              9              0  
2                   5              7             10              0  
3                   6              9             11              1  
4                   8              9             11              1  
...               ...            ...            ...            ...  
52926              18             53             91             12  
52927              15             44             78              4  
52928               5             15             36              0  
52929               3              7             19              0  
52930               2              3             11              0  

[52931 rows x 39 columns]

In [55]:
alphafold_accessibility_smooth = get_smooth_score(
    alphafold_accessibility, 
    np.array(['nAA_2_180_pae', 'nAA_3_180_pae', 'nAA_4_180_pae', 'nAA_4.5_180_pae', 'nAA_5_180_pae', 'nAA_5.5_180_pae', 'nAA_6_180_pae', 'nAA_6.5_180_pae', 'nAA_7_180_pae', 'nAA_7.5_180_pae', 'nAA_8_180_pae','nAA_12_180_pae', 'nAA_18_180_pae', 'nAA_24_180_pae', 'nAA_12_70_pae']), 
    [10])
alphafold_accessibility_smooth;

100%|██████████| 113/113 [00:00<00:00, 346.21it/s]


In [56]:
alphafold_accessibility_smooth['IDR'] = np.where(
    alphafold_accessibility_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)
alphafold_accessibility_smooth

protein_id  protein_number AA  position  quality  x_coord_c  x_coord_ca  \
0       O35129               1  M         1    45.38    -35.172     -33.707   
1       O35129               1  A         2    50.78    -38.162     -37.117   
2       O35129               1  Q         3    52.40    -38.996     -38.671   
3       O35129               1  N         4    51.23    -39.258     -38.274   
4       O35129               1  L         5    50.02    -42.031     -40.574   
..         ...             ... ..       ...      ...        ...         ...   
428     Q9Z2I8             113  S       429    94.38     26.176      27.466   
429     Q9Z2I8             113  V       430    91.54     24.398      24.087   
430     Q9Z2I8             113  A       431    73.79     23.429      23.961   
431     Q9Z2I8             113  K       432    62.38     22.744      23.748   
432     Q9Z2I8             113  K       433    51.30     20.532      20.426   

     x_coord_cb  x_coord_n  y_coord_c  ...  nAA_6_180_pae_smooth10  \
0       -32.901    -33.031      3.159  ...                1.909091   
1       -37.283    -35.709      2.229  ...                1.916667   
2       -38.089    -37.769      4.596  ...                1.923077   
3       -36.904    -38.080      7.104  ...                1.857143   
4       -40.297    -39.604      6.273  ...                1.800000   
..          ...        ...        ...  ...                     ...   
428      27.350     27.774     14.096  ...                2.133333   
429      23.021     25.313     16.707  ...                2.071429   
430      23.247     23.808     19.412  ...                2.076923   
431      24.902     24.238     22.331  ...                2.083333   
432      19.015     21.457     24.555  ...                1.909091   

     nAA_6.5_180_pae_smooth10  nAA_7_180_pae_smooth10  \
0                    1.909091                1.909091   
1                    1.916667                1.916667   
2                    1.923077                1.923077   
3                    1.857143                1.928571   
4                    1.800000                1.933333   
..                        ...                     ...   
428                  3.533333                5.133333   
429                  3.571429                5.214286   
430                  3.538462                5.230769   
431                  3.500000                5.083333   
432                  3.272727                4.909091   

     nAA_7.5_180_pae_smooth10  nAA_8_180_pae_smooth10  \
0                    1.909091                1.909091   
1                    1.916667                1.916667   
2                    1.923077                1.923077   
3                    1.928571                1.928571   
4                    1.933333                1.933333   
..                        ...                     ...   
428                  6.333333                7.000000   
429                  6.428571                7.142857   
430                  6.461538                7.153846   
431                  6.250000                7.000000   
432                  5.909091                6.636364   

     nAA_12_180_pae_smooth10  nAA_18_180_pae_smooth10 nAA_24_180_pae_smooth10  \
0                   6.000000                 8.727273               11.181818   
1                   5.916667                 8.666667               11.416667   
2                   5.769231                 8.538462               11.538462   
3                   5.642857                 8.500000               11.571429   
4                   5.466667                 8.466667               11.733333   
..                       ...                      ...                     ...   
428                20.666667                47.266667               85.200000   
429                20.285714                46.357143               83.642857   
430                20.307692                46.153846               83.307692   
431                20.333333      

# Merge Dataframes into Full Dataset (Includes Alphafold) - Labeled Methionines

In [57]:
alphafold_accessibility_smooth["position"] = alphafold_accessibility_smooth["position"] - 1 # zero-index the positions to match initial dataframe

peptides_wa = peptides_cs.merge(
    alphafold_accessibility_smooth, 
    how="left", 
    left_on=["Protein ID", "Methionine Location"], 
    right_on=["protein_id", "position"]
)
peptides_wa # wa means "with alphafold"

Peptide Sequence      Modified Peptide  \
0            AADTIGYPVMIR          AADTIGYPVMIR   
1              IAMQTLDMGR            IAMQTLDMGR   
2             FVGAVDPIMEK           FVGAVDPIMEK   
3    QAQYLGMPINGPFKPDHYRY  QAQYLGMPINGPFKPDHYRY   
4            FADVIPMNLPHR          FADVIPMNLPHR   
..                    ...                   ...   
198     AHMVTLDYTVQVPGTGR     AHMVTLDYTVQVPGTGR   
199            KEQESEVDMK            KEQESEVDMK   
200       RGVMLAVDAVIAELK       RGVMLAVDAVIAELK   
201           MQLLEIITTDK           MQLLEIITTDK   
202              DHFEEAMR              DHFEEAMR   

             Light Modified Peptide          Heavy Modified Peptide  \
0            AADTIGYPVM[649.3660]IR          AADTIGYPVM[655.3735]IR   
1              IAM[649.3660]QTLDMGR            IAM[655.3735]QTLDMGR   
2             FVGAVDPIM[649.3660]EK           FVGAVDPIM[655.3735]EK   
3    QAQYLGM[649.3660]PINGPFKPDHYRY  QAQYLGM[655.3735]PINGPFKPDHYRY   
4            FADVIPM[649.3660]NLPHR          FADVIPM[655.3735]NLPHR   
..                              ...                             ...   
198     AHM[649.3660]VTLDYTVQVPGTGR     AHM[655.3735]VTLDYTVQVPGTGR   
199            KEQESEVDM[649.3660]K            KEQESEVDM[655.3735]K   
200       RGVM[649.3660]LAVDAVIAELK       RGVM[655.3735]LAVDAVIAELK   
201           M[649.3660]QLLEIITTDK           M[655.3735]QLLEIITTDK   
202              DHFEEAM[649.3660]R              DHFEEAM[655.3735]R   

     1 Log2 Ratio HL  10 Log2 Ratio HL  11 Log2 Ratio HL  12 Log2 Ratio HL  \
0                NaN               NaN          3.269016               NaN   
1                NaN          2.783695          3.114945          2.697822   
2                NaN               NaN               NaN               NaN   
3           2.394458          2.380664          2.682897               NaN   
4                NaN               NaN          2.580647               NaN   
..               ...               ...               ...               ...   
198        -1.822067         -1.687501         -1.706146         -1.647231   
199        -1.759694               NaN         -1.821773         -1.960428   
200        -1.812349         -1.980371         -2.307386         -2.492858   
201        -2.286954         -2.700322         -1.941018         -1.860306   
202              NaN         -2.891986         -2.129614               NaN   

     2 Log2 Ratio HL  3 Log2 Ratio HL  ...  nAA_6_180_pae_smooth10  \
0                NaN              NaN  ...                2.380952   
1                NaN              NaN  ...                2.761905   
2                NaN         2.383482  ...                2.238095   
3           2.435014         2.412394  ...                2.095238   
4           2.446614         2.450718  ...                2.857143   
..               ...              ...  ...                     ...   
198        -1.793966        -1.407199  ...                2.523810   
199        -1.781592        -1.988603  ...                2.000000   
200        -1.405837        -2.094407  ...                2.857143   
201        -2.069488              NaN  ...                2.571429   
202              NaN              NaN  ...                2.285714   

     nAA_6.5_180_pae_smooth10  nAA_7_180_pae_smooth10  \
0                    3.666667                4.857143   
1                    4.857143                6.285714   
2                    3.285714                4.380952   
3                    3.238095                4.428571   
4                    4.238095                6.095238   
..                        ...                     ...   
198                  3.380952                4.428571   
199                  2.904762                3.238095   
200                  5.285714                6.238095   
201                  3.428571                4.904762   
202                  3.476190                4.190476   

     nAA_7.5_180_pae_smooth10  nAA_8_180_pae_smooth10  \
0                    6.

In [58]:
#peptides_wa.to_csv(os.path.join(curr_dir_path, "Mitochondria_peptides_with_alphafold.csv"))

In [59]:
path = os.path.join(curr_dir_path, "Mitochondria_peptides_with_alphafold.csv")
peptides_wa = pd.read_csv(path)
peptides_wa.set_index("Unnamed: 0", inplace=True)
peptides_wa.index.name = None
peptides_wa

Peptide Sequence      Modified Peptide  \
0            AADTIGYPVMIR          AADTIGYPVMIR   
1              IAMQTLDMGR            IAMQTLDMGR   
2             FVGAVDPIMEK           FVGAVDPIMEK   
3    QAQYLGMPINGPFKPDHYRY  QAQYLGMPINGPFKPDHYRY   
4            FADVIPMNLPHR          FADVIPMNLPHR   
..                    ...                   ...   
198     AHMVTLDYTVQVPGTGR     AHMVTLDYTVQVPGTGR   
199            KEQESEVDMK            KEQESEVDMK   
200       RGVMLAVDAVIAELK       RGVMLAVDAVIAELK   
201           MQLLEIITTDK           MQLLEIITTDK   
202              DHFEEAMR              DHFEEAMR   

             Light Modified Peptide          Heavy Modified Peptide  \
0            AADTIGYPVM[649.3660]IR          AADTIGYPVM[655.3735]IR   
1              IAM[649.3660]QTLDMGR            IAM[655.3735]QTLDMGR   
2             FVGAVDPIM[649.3660]EK           FVGAVDPIM[655.3735]EK   
3    QAQYLGM[649.3660]PINGPFKPDHYRY  QAQYLGM[655.3735]PINGPFKPDHYRY   
4            FADVIPM[649.3660]NLPHR          FADVIPM[655.3735]NLPHR   
..                              ...                             ...   
198     AHM[649.3660]VTLDYTVQVPGTGR     AHM[655.3735]VTLDYTVQVPGTGR   
199            KEQESEVDM[649.3660]K            KEQESEVDM[655.3735]K   
200       RGVM[649.3660]LAVDAVIAELK       RGVM[655.3735]LAVDAVIAELK   
201           M[649.3660]QLLEIITTDK           M[655.3735]QLLEIITTDK   
202              DHFEEAM[649.3660]R              DHFEEAM[655.3735]R   

     1 Log2 Ratio HL  10 Log2 Ratio HL  11 Log2 Ratio HL  12 Log2 Ratio HL  \
0                NaN               NaN          3.269016               NaN   
1                NaN          2.783695          3.114945          2.697822   
2                NaN               NaN               NaN               NaN   
3           2.394458          2.380664          2.682897               NaN   
4                NaN               NaN          2.580647               NaN   
..               ...               ...               ...               ...   
198        -1.822067         -1.687501         -1.706146         -1.647231   
199        -1.759694               NaN         -1.821773         -1.960428   
200        -1.812349         -1.980371         -2.307386         -2.492858   
201        -2.286954         -2.700322         -1.941018         -1.860306   
202              NaN         -2.891986         -2.129614               NaN   

     2 Log2 Ratio HL  3 Log2 Ratio HL  ...  nAA_6_180_pae_smooth10  \
0                NaN              NaN  ...                2.380952   
1                NaN              NaN  ...                2.761905   
2                NaN         2.383482  ...                2.238095   
3           2.435014         2.412394  ...                2.095238   
4           2.446614         2.450718  ...                2.857143   
..               ...              ...  ...                     ...   
198        -1.793966        -1.407199  ...                2.523810   
199        -1.781592        -1.988603  ...                2.000000   
200        -1.405837        -2.094407  ...                2.857143   
201        -2.069488              NaN  ...                2.571429   
202              NaN              NaN  ...                2.285714   

     nAA_6.5_180_pae_smooth10  nAA_7_180_pae_smooth10  \
0                    3.666667                4.857143   
1                    4.857143                6.285714   
2                    3.285714                4.380952   
3                    3.238095                4.428571   
4                    4.238095                6.095238   
..                        ...                     ...   
198                  3.380952                4.428571   
199                  2.904762                3.238095   
200                  5.285714                6.238095   
201                  3.428571                4.904762   
202                  3.476190                4.190476   

     nAA_7.5_180_pae_smooth10  nAA_8_180_pae_smooth10  \
0                    6.

# Load Dataset (MitoCarta3.0) - Full Mitochondrial Proteome

In [60]:
data_loc = os.path.join(curr_dir_path, "Mouse.MitoCarta3.0.xls")
mitocarta3_0 = pd.read_excel(data_loc, sheet_name="A Mouse MitoCarta3.0")
mitocarta3_0

MouseGeneID  HumanOrthologGeneID   Symbol  \
0           66445               1537.0     Cyc1   
1           18597               5160.0    Pdha1   
2           66043                513.0    Atp5d   
3           74316             122961.0    Isca2   
4           68263               5162.0     Pdhb   
...           ...                  ...      ...   
1135        71703              51566.0   Armcx3   
1136        67474               9342.0   Snap29   
1137        18970               5423.0     Polb   
1138    109729085          109703458.0     Htd2   
1139        66364          101928527.0  Pigbos1   

                                               Synonyms  \
0                          2610002H19Rik|AA408921|Cyct1   
1                                           Pdha|Pdha-1   
2     0610008F14Rik|1500000I11Rik|AA960090|AI876556|...   
3                0710001C05Rik|5730594E03Rik|Hbld|Hbld1   
4                         2610103L06Rik|AL024199|C81408   
...                                                 ...   
1135                       1200004E24Rik|AI450003|ALEX3   
1136      1300018G05Rik|AI891940|AU020222|BB131856|Gs32   
1137                                      A430088C08Rik   
1138                                                  -   
1139                        2310009A05Rik|5031420N21Rik   

                                            Description MitoCarta3.0_List  \
0                                        cytochrome c-1      MitoCarta3.0   
1                     pyruvate dehydrogenase E1 alpha 1      MitoCarta3.0   
2     ATP synthase, H+ transporting, mitochondrial F...      MitoCarta3.0   
3                        iron-sulfur cluster assembly 2      MitoCarta3.0   
4               pyruvate dehydrogenase (lipoamide) beta      MitoCarta3.0   
...                                                 ...               ...   
1135            armadillo repeat containing, X-linked 3      MitoCarta3.0   
1136                 synaptosomal-associated protein 29      MitoCarta3.0   
1137                    polymerase (DNA directed), beta      MitoCarta3.0   
1138           hydroxyacyl-thioester dehydratase type 2      MitoCarta3.0   
1139                             Pigb opposite strand 1      MitoCarta3.0   

                                  MitoCarta3.0_Evidence  \
0     literature, APEX_IMS, APEX_matrix, targetP sig...   
1     literature, APEX_matrix, targetP signal+, yeas...   
2     literature, APEX_matrix, targetP signal, yeast...   
3     APEX_matrix, targetP signal+, yeast mito homol...   
4     literature, targetP signal+, yeast mito homolo...   
...                                                 ...   
1135                                         literature   
1136                                           APEX_IMS   
1137                                         literature   
1138                                         literature   
1139                                         literature   

     MitoCarta3.0_SubMitoLocalization  \
0                                 MIM   
1                              Matrix   
2                                 MIM   
3                              Matrix   
4                              Matrix   
...                               ...   
1135                              MOM   
1136                              IMS   
1137                           Matrix   
1138                           Matrix   
1139                              MOM   

                              MitoCarta3.0_MitoPathways  \
0     OXPHOS > Complex III > CIII subunits | Metabol...   
1     Metabolism > Carbohydrate metabolism > Pyruvat...   
2     OXPHOS > Complex V > CV subunits | OXPHOS > OX...   
3     Metabolism > Metals and cofactors > Fe-S clust...   
4     Metabolism > Carbohydrate metabolism > Pyruvat...   
...                                                 ...   
1135  Mitochondrial dynamics and surveillance > Traf...   
1136  Mitochondrial dynamics and surveillance > Auto...   
1137  Mitochondrial centra

In [61]:
# Calculate number of proteins in the mitochondrial proteome
is_mitochondrial = (mitocarta3_0["HPA_Main_Location_2020 (Reliability)"].str.contains("mitoch", case=False))
(is_mitochondrial == True).value_counts(dropna=False)

HPA_Main_Location_2020 (Reliability)
False    609
True     531
Name: count, dtype: int64

In [62]:
# Sanity Check - ensure protein split was done correctly (correct mitoch... string matching)

#pd.set_option('display.max_rows', None)
##display(mitocarta3_0[is_mitochondrial == True]["HPA_Main_Location_2020 (Reliability)"].value_counts())
#pd.reset_option('display.max_rows')

In [63]:
# Filter MitoCarta3.0 dataset to only include mitochondrial proteins
mitocarta3_0_mitochondrial = mitocarta3_0[is_mitochondrial == True]
mitocarta3_0_mitochondrial;

In [64]:
# Drop rows with NaN UniProt IDs (just one)
print(mitocarta3_0_mitochondrial["UniProt"].isna().value_counts())
mitocarta3_0_mitochondrial = mitocarta3_0_mitochondrial.dropna(subset=["UniProt"])
mitocarta3_0_mitochondrial;

UniProt
False    530
True       1
Name: count, dtype: int64


In [65]:
unique_uniprotIDs = mitocarta3_0_mitochondrial["UniProt"].unique()
print("Unique UniProt IDs: \n" + str(unique_uniprotIDs))
print("Number of Unique UniProt IDs: " + str(unique_uniprotIDs.size))

Unique UniProt IDs: 
['Q9D0M3' 'P35486' 'Q9D051' 'Q9CZ13' 'Q9DB15' 'Q8BJZ4' 'Q9CR68' 'Q8K2B3'
 'Q9CQA3' 'Q03265' 'Q8BTE0' 'Q9CXT8' 'Q9DB77' 'Q60597' 'Q9CQ69' 'P08249'
 'P53395' 'Q8BMF4' 'Q6PB66' 'Q9D1H8' 'Q8K0D5' 'Q99LP6' 'Q8BMS4' 'Q8VEM8'
 'Q9DC71' 'P56480' 'Q8VE22' 'Q99LC5' 'P97807' 'Q8BK72' 'Q9D6R2' 'Q91VD9'
 'Q8JZQ2' 'Q9DC61' 'Q9Z2I9' 'Q8VDC0' 'Q9DB20' 'Q9EQI8' 'Q8BJ03' 'Q9CQJ1'
 'P62073' 'Q8BH95' 'Q8BMS1' 'Q9WUM5' 'Q9CZU6' 'P43024' 'Q07417' 'Q9D6J6'
 'Q8BIJ6' 'Q06185' 'Q91VA7' 'Q9Z2I0' 'Q5RL20' 'Q8K4F5' 'Q9D880' 'Q9D1B9'
 'Q8C6I2' 'Q9D6J5' 'O08749' 'Q91WD5' 'Q91YT0' 'P58281' 'Q8CAQ8' 'Q78IK4'
 'Q8CGK3' 'Q924L1' 'Q91YJ5' 'Q8K3J1' 'Q9D773' 'Q8R2Q4' 'Q9WV98' 'Q8K411'
 'P45952' 'Q80YD1' 'Q99N96' 'Q99KI0' 'Q8BGH2' 'Q924T2' 'Q9CY73' 'P67778'
 'Q8C3X4' 'Q8BKF1' 'Q91VC9' 'P62075' 'P52825' 'Q3TBW2' 'P97450' 'Q8CAK1'
 'Q9Z2Q5' 'Q8QZS1' 'Q9D0K2' 'Q9D172' 'Q9CQJ8' 'Q9CPQ1' 'P09671' 'Q9CQH3'
 'Q61425' 'Q99M04' 'P46656' 'Q99J25' 'Q9CWV0' 'Q99JT1' 'Q8BYL4' 'O35943'
 'P70404' 'Q9CY16' 'O35857' 'Q

In [66]:
# Load and update sequence cache df: mapping from UniProt IDs to complete AA sequence
# SLOW - ONLY DO THIS ONCE - CONVERT TO CSV FILE, THEN RE-LOAD FROM THAT FILE

#path = os.path.join(global_data_path, "complete_sequence_cache.csv")
#sequence_cache_df = pd.read_csv(path)
#sequence_cache_df.set_index("Unnamed: 0", inplace=True)
#sequence_cache_df.index.name = None
#display(sequence_cache_df)
#
## Determine unknown sequences
#
#unknown_uniprotIDs_idxs = ~np.isin(unique_uniprotIDs, sequence_cache_df["Protein ID"].values)
#unknown_uniprotIDs = unique_uniprotIDs[unknown_uniprotIDs_idxs]
#unknown_sequences_df = pd.DataFrame({"Protein ID": unknown_uniprotIDs})
#display(unknown_sequences_df)
#
## Retrieve unknown sequences
#
#tqdm.pandas()
#unknown_sequences_df["Complete Sequence"] = unknown_sequences_df["Protein ID"].progress_apply(get_complete_sequence)
#display(unknown_sequences_df)
#
#sequence_cache_df_updated = pd.concat([sequence_cache_df, unknown_sequences_df])
#sequence_cache_df_updated.to_csv(os.path.join(global_data_path, "complete_sequence_cache.csv"))
#sequence_cache_df_updated;

In [67]:
# Load cache df: mapping from UniProt IDs to complete AA sequence
path = os.path.join(global_data_path, "complete_sequence_cache.csv")
sequence_cache_df_updated = pd.read_csv(path)
sequence_cache_df_updated.set_index("Unnamed: 0", inplace=True)
sequence_cache_df_updated.index.name = None
sequence_cache_df_updated;

In [68]:
mitocarta3_0_mitochondrial_cs = mitocarta3_0_mitochondrial.merge(sequence_cache_df_updated, how="left", left_on="UniProt", right_on="Protein ID")
mitocarta3_0_mitochondrial_cs.index = mitocarta3_0_mitochondrial.index
mitocarta3_0_mitochondrial_cs = mitocarta3_0_mitochondrial_cs.drop(columns=["Protein ID"])
mitocarta3_0_mitochondrial_cs

MouseGeneID  HumanOrthologGeneID  Symbol  \
0           66445               1537.0    Cyc1   
1           18597               5160.0   Pdha1   
4           68263               5162.0    Pdhb   
5           22273               7384.0  Uqcrc1   
7           56282               6182.0  Mrpl12   
...           ...                  ...     ...   
1123        67759              55848.0  Plgrkt   
1129        78248              51309.0  Armcx1   
1130        67416               9823.0  Armcx2   
1132       269642             339983.0   Nat8l   
1134        76781              64863.0  Mettl4   

                                               Synonyms  \
0                          2610002H19Rik|AA408921|Cyct1   
1                                           Pdha|Pdha-1   
4                         2610103L06Rik|AL024199|C81408   
5                                         1110032G10Rik   
7     0610034O11Rik|1500031N16Rik|L12mt|MRP-|MRP-L12...   
...                                                 ...   
1123  1110007H22Rik|5033414D02Rik|AI852040|Plg-|Plg-...   
1129                                3010033I09Rik|ALEX1   
1130                       3230401N03Rik|AI043003|ALEX2   
1132                             1110038O08Rik|Sh|Shati   
1134  2410198H06Rik|A730091E08Rik|AV296509|HsT66|HsT661   

                                            Description MitoCarta3.0_List  \
0                                        cytochrome c-1      MitoCarta3.0   
1                     pyruvate dehydrogenase E1 alpha 1      MitoCarta3.0   
4               pyruvate dehydrogenase (lipoamide) beta      MitoCarta3.0   
5       ubiquinol-cytochrome c reductase core protein 1      MitoCarta3.0   
7                   mitochondrial ribosomal protein L12      MitoCarta3.0   
...                                                 ...               ...   
1123  plasminogen receptor, C-terminal lysine transm...      MitoCarta3.0   
1129            armadillo repeat containing, X-linked 1      MitoCarta3.0   
1130            armadillo repeat containing, X-linked 2      MitoCarta3.0   
1132                         N-acetyltransferase 8-like      MitoCarta3.0   
1134                           methyltransferase like 4      MitoCarta3.0   

                                  MitoCarta3.0_Evidence  \
0     literature, APEX_IMS, APEX_matrix, targetP sig...   
1     literature, APEX_matrix, targetP signal+, yeas...   
4     literature, targetP signal+, yeast mito homolo...   
5     literature, APEX_matrix, targetP signal+, yeas...   
7     literature, targetP signal+, yeast mito homolo...   
...                                                 ...   
1123                                                GFP   
1129                                         literature   
1130                                         literature   
1132                                         literature   
1134                                         literature   

     MitoCarta3.0_SubMitoLocalization  \
0                                 MIM   
1                              Matrix   
4                              Matrix   
5                                 MIM   
7                              Matrix   
...                               ...   
1123                         Membrane   
1129                              MOM   
1130                         Membrane   
1132                         Membrane   
1134                           Matrix   

                              MitoCarta3.0_MitoPathways TrainingDataset  ...  \
0     OXPHOS > Complex III > CIII subunits | Metabol...           Tmito  ...   
1     Metabolism > Carbohydrate metabolism > Pyruvat...           Tmito  ...   
4     Metabolism > Carbohydrate metabolism > Pyruvat...           Tmito  ...   
5     Protein import, sorting and homeostasis > Prot...           Tmito  ...   
7     Mitochondrial central dogma > mtRNA metabolism...           Tmito  ...   
...                                                 ...             ...  ...   
1123     

In [69]:
# NOTE: Sequence length from dataset doesn't exactly match up with length of sequence as determined by UniProtID - weird
(mitocarta3_0_mitochondrial_cs["ProteinLength"] - mitocarta3_0_mitochondrial_cs["Complete Sequence"].str.len()).value_counts();

# Download Alphafold Data - Full Mitochondrial Proteome

In [70]:
# Path for alphafold protein data

alphafold_path_str = "../alphafold_data/"
alphafold_path = os.path.abspath(alphafold_path_str)

cif_dir = os.path.join(alphafold_path, "cif")
pae_dir = os.path.join(alphafold_path, "pae")

print(alphafold_path)
print(cif_dir)
print(pae_dir)

/Users/ritwiksrinivas/Desktop/Projects/MetML/alphafold_data
/Users/ritwiksrinivas/Desktop/Projects/MetML/alphafold_data/cif
/Users/ritwiksrinivas/Desktop/Projects/MetML/alphafold_data/pae


In [71]:
# NOTE: these IDs are invalid from an Alphafold perspective - they are the secondary UniProtIDs, which was fine for querying UniProt, but not Alphafold
# so, manually impute these IDs with their primary ones
# invalid_proteins_cif -> ['F7C846', 'Q9JLT4', 'Q3UW66', 'Q91XR9']

replace_dict = {'F7C846': 'Q8R5C0', 'Q3UW66': 'Q99J99', 'Q91XR9': 'O70325'} 
# Q9JLT4 is weird - seems to be primary ID, but not in Alphafold
# Q91XR9 / O70325 is weird - O70325 exists in UniProt, but not Alphafold

mitocarta3_0_mitochondrial_cs["UniProt-Primary"] = mitocarta3_0_mitochondrial_cs["UniProt"].replace(replace_dict)
mitocarta3_0_mitochondrial_cs = mitocarta3_0_mitochondrial_cs.drop(mitocarta3_0_mitochondrial_cs[mitocarta3_0_mitochondrial_cs["UniProt"] == "Q9JLT4"].index)
mitocarta3_0_mitochondrial_cs = mitocarta3_0_mitochondrial_cs.drop(mitocarta3_0_mitochondrial_cs[mitocarta3_0_mitochondrial_cs["UniProt-Primary"] == "O70325"].index)
mitocarta3_0_mitochondrial_cs;

In [72]:
# Set uniprot IDs to use

uniprotIDs_fullproteome = mitocarta3_0_mitochondrial_cs["UniProt-Primary"].unique()
print("Unique UniProt IDs: \n" + str(uniprotIDs_fullproteome))
print("Number of Unique UniProt IDs: " + str(uniprotIDs_fullproteome.size))

Unique UniProt IDs: 
['Q9D0M3' 'P35486' 'Q9D051' 'Q9CZ13' 'Q9DB15' 'Q8BJZ4' 'Q9CR68' 'Q8K2B3'
 'Q9CQA3' 'Q03265' 'Q8BTE0' 'Q9CXT8' 'Q9DB77' 'Q60597' 'Q9CQ69' 'P08249'
 'P53395' 'Q8BMF4' 'Q6PB66' 'Q9D1H8' 'Q8K0D5' 'Q99LP6' 'Q8BMS4' 'Q8VEM8'
 'Q9DC71' 'P56480' 'Q8VE22' 'Q99LC5' 'P97807' 'Q8BK72' 'Q9D6R2' 'Q91VD9'
 'Q8JZQ2' 'Q9DC61' 'Q9Z2I9' 'Q8VDC0' 'Q9DB20' 'Q9EQI8' 'Q8BJ03' 'Q9CQJ1'
 'P62073' 'Q8BH95' 'Q8BMS1' 'Q9WUM5' 'Q9CZU6' 'P43024' 'Q07417' 'Q9D6J6'
 'Q8BIJ6' 'Q06185' 'Q91VA7' 'Q9Z2I0' 'Q5RL20' 'Q8K4F5' 'Q9D880' 'Q9D1B9'
 'Q8C6I2' 'Q9D6J5' 'O08749' 'Q91WD5' 'Q91YT0' 'P58281' 'Q8CAQ8' 'Q78IK4'
 'Q8CGK3' 'Q924L1' 'Q91YJ5' 'Q8K3J1' 'Q9D773' 'Q8R2Q4' 'Q9WV98' 'Q8K411'
 'P45952' 'Q80YD1' 'Q99N96' 'Q99KI0' 'Q8BGH2' 'Q924T2' 'Q9CY73' 'P67778'
 'Q8C3X4' 'Q8BKF1' 'Q91VC9' 'P62075' 'P52825' 'Q3TBW2' 'P97450' 'Q8CAK1'
 'Q9Z2Q5' 'Q8QZS1' 'Q9D0K2' 'Q9D172' 'Q9CQJ8' 'Q9CPQ1' 'P09671' 'Q9CQH3'
 'Q61425' 'Q99M04' 'P46656' 'Q99J25' 'Q9CWV0' 'Q99JT1' 'Q8BYL4' 'O35943'
 'P70404' 'Q9CY16' 'O35857' 'Q

In [73]:
# Download cif data for proteins
# SLOW THE FIRST TIME - caches the relevant cif data
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=uniprotIDs_fullproteome,
    out_folder=cif_dir
)

100%|██████████| 528/528 [00:00<00:00, 192858.36it/s]

2024-10-17 15:34:59> Valid proteins: 0
2024-10-17 15:34:59> Invalid proteins: 0
2024-10-17 15:34:59> Existing proteins: 528


In [74]:
# download pae data for proteins
# SLOW THE FIRST TIME - caches the relevant pae data
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=uniprotIDs_fullproteome,
    out_folder=pae_dir, 
)

100%|██████████| 528/528 [00:00<00:00, 209022.42it/s]

2024-10-17 15:34:59> Valid proteins: 0
2024-10-17 15:34:59> Invalid proteins: 0
2024-10-17 15:34:59> Existing proteins: 528


# Construct Alphafold Dataframe (Calculate Accessibilities) - Full Mitochondrial Proteome

In [75]:
# Format alphafold data into dataframe

alphafold_annotation_full = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=uniprotIDs_fullproteome)
alphafold_annotation_full

100%|██████████| 1630/1630 [00:31<00:00, 52.26it/s] 


protein_id  protein_number AA  position  quality  x_coord_c  \
0          A2ADF7               1  M         1    34.90    -16.170   
1          A2ADF7               1  K         2    27.13    -16.032   
2          A2ADF7               1  P         3    32.83    -16.958   
3          A2ADF7               1  T         4    33.64    -16.686   
4          A2ADF7               1  Q         5    28.86    -16.698   
...           ...             ... ..       ...      ...        ...   
197263     S4R2K0             528  M       227    93.01     -4.021   
197264     S4R2K0             528  E       228    93.81     -7.039   
197265     S4R2K0             528  V       229    88.77     -7.847   
197266     S4R2K0             528  N       230    90.23     -6.518   
197267     S4R2K0             528  D       231    84.67     -8.280   

        x_coord_ca  x_coord_cb  x_coord_n  y_coord_c  ...  z_coord_ca  \
0          -15.807     -16.898    -15.637    -29.957  ...      38.096   
1          -15.462     -14.173    -15.203    -26.991  ...      38.724   
2          -17.790     -19.230    -17.200    -24.520  ...      36.289   
3          -15.841     -15.615    -16.546    -22.219  ...      33.884   
4          -16.784     -16.026    -16.209    -19.372  ...      34.618   
...            ...         ...        ...        ...  ...         ...   
197263      -2.734      -1.530     -2.851      2.349  ...     -27.747   
197264      -5.845      -6.164     -4.573      1.709  ...     -27.713   
197265      -7.972      -7.895     -6.944      4.796  ...     -27.292   
197266      -6.363      -4.940     -6.637      4.882  ...     -30.321   
197267      -6.767      -6.194     -6.463      2.565  ...     -32.605   

        z_coord_cb  z_coord_n  secondary_structure  structure_group  BEND  \
0           37.222     39.345         unstructured     unstructured     0   
1           39.162     38.463         unstructured     unstructured     0   
2           36.675     37.473         unstructured     unstructured     0   
3           32.424     34.597         unstructured     unstructured     0   
4           35.575     34.658         unstructured     unstructured     0   
...            ...        ...                  ...              ...   ...   
197263     -27.920    -26.354         HELX_RH_AL_P             HELX     0   
197264     -26.580    -27.423         HELX_RH_AL_P             HELX     0   
197265     -26.038    -27.216         HELX_RH_AL_P             HELX     0   
197266     -30.226    -29.116         HELX_RH_AL_P             HELX     0   
197267     -32.255    -31.516         unstructured     unstructured     0   

        HELX STRN TURN  unstructured  
0          0    0    0             1  
1          0    0    0             1  
2          0    0    0             1  
3          0    0    0             1  
4          0    0    0             1  
...      ...  ...  ...           ...  
197263     1    0    0             0  
197264     1    0    0             0  
197265     1    0    0             0  
197266     1    0    0             0  
197267     0    0    0             1  

[197268 rows x 24 columns]

In [76]:
# Calculate full sphere exposure -> radius = 2
exposure_sphere_rad2 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=2, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad2;

100%|██████████| 528/528 [00:02<00:00, 211.84it/s]


In [77]:
alphafold_accessibility_full = alphafold_annotation_full.merge(
    exposure_sphere_rad2, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [78]:
# Calculate full sphere exposure -> radius = 3
exposure_sphere_rad3 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=3, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad3;

100%|██████████| 528/528 [00:02<00:00, 261.40it/s]


In [79]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad3, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [80]:
# Calculate full sphere exposure -> radius = 4
exposure_sphere_rad4 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=4, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4;

100%|██████████| 528/528 [00:02<00:00, 214.44it/s]


In [81]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad4, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [82]:
# Calculate full sphere exposure -> radius = 4.5
exposure_sphere_rad4_5 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=4.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4_5;

100%|██████████| 528/528 [00:02<00:00, 239.07it/s]


In [83]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad4_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [84]:
# Calculate full sphere exposure -> radius = 5
exposure_sphere_rad5 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5;

100%|██████████| 528/528 [00:02<00:00, 251.21it/s]


In [85]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [86]:
# Calculate full sphere exposure -> radius = 5.5
exposure_sphere_rad5_5 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=5.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5_5;

100%|██████████| 528/528 [00:02<00:00, 228.28it/s]


In [87]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad5_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [88]:
# Calculate full sphere exposure -> radius = 6
exposure_sphere_rad6 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=6, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6;

100%|██████████| 528/528 [00:02<00:00, 220.06it/s]


In [89]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad6, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [90]:
# Calculate full sphere exposure -> radius = 6.5
exposure_sphere_rad6_5 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=6.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6_5;

100%|██████████| 528/528 [00:02<00:00, 240.28it/s]


In [91]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad6_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [92]:
# Calculate full sphere exposure -> radius = 7
exposure_sphere_rad7 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=7, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7;

100%|██████████| 528/528 [00:02<00:00, 217.47it/s]


In [93]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad7, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [94]:
# Calculate full sphere exposure -> radius = 7.5
exposure_sphere_rad7_5 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=7.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7_5;

100%|██████████| 528/528 [00:02<00:00, 235.28it/s]


In [95]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad7_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [96]:
# Calculate full sphere exposure -> radius = 8
exposure_sphere_rad8 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=8, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad8;

100%|██████████| 528/528 [00:02<00:00, 233.72it/s]


In [97]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad8, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [98]:
# Calculate full sphere exposure -> radius = 12
exposure_sphere_rad12 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=12, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad12;

100%|██████████| 528/528 [00:02<00:00, 197.88it/s]


In [99]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [100]:
# Calculate full sphere exposure -> radius = 18
exposure_sphere_rad18 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=18, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad18;

100%|██████████| 528/528 [00:04<00:00, 128.60it/s]


In [101]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad18, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [102]:
# Calculate full sphere exposure -> radius = 24
exposure_sphere_rad24 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad24;

100%|██████████| 528/528 [00:05<00:00, 93.27it/s] 


In [103]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_sphere_rad24, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full;

In [104]:
# Calculate part sphere exposure -> angle = 70, radius = 12
exposure_ang70_rad12 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)
exposure_ang70_rad12;

100%|██████████| 528/528 [00:02<00:00, 181.71it/s]


In [105]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    exposure_ang70_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full

protein_id  protein_number AA  position  quality  x_coord_c  \
0          A2ADF7               1  M         1    34.90    -16.170   
1          A2ADF7               1  K         2    27.13    -16.032   
2          A2ADF7               1  P         3    32.83    -16.958   
3          A2ADF7               1  T         4    33.64    -16.686   
4          A2ADF7               1  Q         5    28.86    -16.698   
...           ...             ... ..       ...      ...        ...   
197263     S4R2K0             528  M       227    93.01     -4.021   
197264     S4R2K0             528  E       228    93.81     -7.039   
197265     S4R2K0             528  V       229    88.77     -7.847   
197266     S4R2K0             528  N       230    90.23     -6.518   
197267     S4R2K0             528  D       231    84.67     -8.280   

        x_coord_ca  x_coord_cb  x_coord_n  y_coord_c  ...  nAA_5.5_180_pae  \
0          -15.807     -16.898    -15.637    -29.957  ...                1   
1          -15.462     -14.173    -15.203    -26.991  ...                1   
2          -17.790     -19.230    -17.200    -24.520  ...                2   
3          -15.841     -15.615    -16.546    -22.219  ...                1   
4          -16.784     -16.026    -16.209    -19.372  ...                1   
...            ...         ...        ...        ...  ...              ...   
197263      -2.734      -1.530     -2.851      2.349  ...                2   
197264      -5.845      -6.164     -4.573      1.709  ...                2   
197265      -7.972      -7.895     -6.944      4.796  ...                2   
197266      -6.363      -4.940     -6.637      4.882  ...                2   
197267      -6.767      -6.194     -6.463      2.565  ...                0   

        nAA_6_180_pae  nAA_6.5_180_pae  nAA_7_180_pae  nAA_7.5_180_pae  \
0                   1                1              1                1   
1                   2                2              2                2   
2                   2                2              2                2   
3                   2                2              2                2   
4                   2                2              2                2   
...               ...              ...            ...              ...   
197263              2                2              2                3   
197264              2                2              2                2   
197265              2                2              2                2   
197266              2                2              2                2   
197267              1                1              1                1   

        nAA_8_180_pae  nAA_12_180_pae nAA_18_180_pae nAA_24_180_pae  \
0                   1               2              3              4   
1                   2               3              4              6   
2                   2               4              5              6   
3                   2               4              6              8   
4                   2               4              6              9   
...               ...             ...            ...            ...   
197263              3               6              9             15   
197264              3               6              8             13   
197265              3               5              8             11   
197266              2               5              6              9   
197267              1               3              5              6   

        nAA_12_70_pae  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  
...               ...  
197263              0  
197264              1  
197265              1  
197266              2  
197267              1  

[197268 rows x 39 columns]

In [106]:
alphafold_accessibility_full_smooth = get_smooth_score(
    alphafold_accessibility_full, 
    np.array(['nAA_2_180_pae', 'nAA_3_180_pae', 'nAA_4_180_pae', 'nAA_4.5_180_pae', 'nAA_5_180_pae', 'nAA_5.5_180_pae', 'nAA_6_180_pae', 'nAA_6.5_180_pae', 'nAA_7_180_pae', 'nAA_7.5_180_pae', 'nAA_8_180_pae','nAA_12_180_pae', 'nAA_18_180_pae', 'nAA_24_180_pae', 'nAA_12_70_pae']), 
    [10])
alphafold_accessibility_full_smooth;

100%|██████████| 528/528 [00:00<00:00, 659.14it/s]


In [107]:
alphafold_accessibility_full_smooth['IDR'] = np.where(
    alphafold_accessibility_full_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)
alphafold_accessibility_full_smooth

protein_id  protein_number AA  position  quality  x_coord_c  x_coord_ca  \
0       A2ADF7               1  M         1    34.90    -16.170     -15.807   
1       A2ADF7               1  K         2    27.13    -16.032     -15.462   
2       A2ADF7               1  P         3    32.83    -16.958     -17.790   
3       A2ADF7               1  T         4    33.64    -16.686     -15.841   
4       A2ADF7               1  Q         5    28.86    -16.698     -16.784   
..         ...             ... ..       ...      ...        ...         ...   
226     S4R2K0             528  M       227    93.01     -4.021      -2.734   
227     S4R2K0             528  E       228    93.81     -7.039      -5.845   
228     S4R2K0             528  V       229    88.77     -7.847      -7.972   
229     S4R2K0             528  N       230    90.23     -6.518      -6.363   
230     S4R2K0             528  D       231    84.67     -8.280      -6.767   

     x_coord_cb  x_coord_n  y_coord_c  ...  nAA_6_180_pae_smooth10  \
0       -16.898    -15.637    -29.957  ...                1.909091   
1       -14.173    -15.203    -26.991  ...                1.916667   
2       -19.230    -17.200    -24.520  ...                1.923077   
3       -15.615    -16.546    -22.219  ...                1.928571   
4       -16.026    -16.209    -19.372  ...                1.933333   
..          ...        ...        ...  ...                     ...   
226      -1.530     -2.851      2.349  ...                2.066667   
227      -6.164     -4.573      1.709  ...                2.071429   
228      -7.895     -6.944      4.796  ...                2.076923   
229      -4.940     -6.637      4.882  ...                2.083333   
230      -6.194     -6.463      2.565  ...                2.090909   

     nAA_6.5_180_pae_smooth10  nAA_7_180_pae_smooth10  \
0                    1.909091                1.909091   
1                    1.916667                1.916667   
2                    1.923077                1.923077   
3                    1.928571                1.928571   
4                    1.933333                1.933333   
..                        ...                     ...   
226                  2.533333                3.333333   
227                  2.571429                3.214286   
228                  2.461538                3.076923   
229                  2.500000                3.000000   
230                  2.363636                2.727273   

     nAA_7.5_180_pae_smooth10  nAA_8_180_pae_smooth10  \
0                    1.909091                1.909091   
1                    1.916667                1.916667   
2                    1.923077                1.923077   
3                    1.928571                1.928571   
4                    1.933333                1.933333   
..                        ...                     ...   
226                  3.866667                4.333333   
227                  3.714286                4.071429   
228                  3.615385                4.000000   
229                  3.583333                4.000000   
230                  3.272727                3.727273   

     nAA_12_180_pae_smooth10  nAA_18_180_pae_smooth10 nAA_24_180_pae_smooth10  \
0                   3.727273                 5.545455                8.454545   
1                   3.750000                 5.666667                8.666667   
2                   3.769231                 5.769231                8.769231   
3                   3.857143                 5.857143                8.714286   
4                   3.866667                 5.866667                8.733333   
..                       ...                      ...                     ...   
226                10.133333                20.333333               38.200000   
227                 9.500000                19.357143               35.785714   
228                 9.153846                18.615385               34.000000   
229                 9.250000      

# Merge Dataframes into Full Dataset (Includes Alphafold) - Full Mitochondrial Proteome

In [108]:
# filter out df to exclude (likely) cleaved N-terminal methionines based on next AA
n_terminal_met_cleave_partners = np.array(["G", "A", "S", "T", "C", "P", "V"])
nextAA = alphafold_accessibility_full_smooth["AA"].shift(periods=-1, fill_value="X")
n_terminal_met_cond = ((alphafold_accessibility_full_smooth["AA"] == "M") & (alphafold_accessibility_full_smooth["position"] == 1) & np.isin(nextAA, n_terminal_met_cleave_partners))
mitocarta3_0_methionines = alphafold_accessibility_full_smooth[~n_terminal_met_cond]
mitocarta3_0_methionines;

In [109]:
# filter out df to only include methionines
mitocarta3_0_methionines = mitocarta3_0_methionines[mitocarta3_0_methionines["AA"] == "M"]
mitocarta3_0_methionines;

In [110]:
mitocarta3_0_methionines["position"] = mitocarta3_0_methionines["position"] - 1 # zero-index the positions to match initial dataframe

mitocarta3_0_methionines_wa = mitocarta3_0_methionines.merge(
    mitocarta3_0_mitochondrial_cs[["UniProt", "UniProt-Primary", "Complete Sequence"]], 
    how="left", 
    left_on="protein_id", 
    right_on="UniProt-Primary"
)
mitocarta3_0_methionines_wa

protein_id  protein_number AA  position  quality  x_coord_c  x_coord_ca  \
0        A2ADF7               1  M         0    34.90    -16.170     -15.807   
1        A2ADF7               1  M         7    25.00    -17.959     -18.801   
2        A2ADF7               1  M        11    31.62    -22.014     -20.790   
3        A2ADF7               1  M        16    60.19    -24.992     -24.872   
4        A2ADF7               1  M        96    86.39    -12.515     -11.617   
...         ...             ... ..       ...      ...        ...         ...   
4492     S4R2K0             528  M        90    98.40     -3.270      -3.856   
4493     S4R2K0             528  M       132    98.22     14.774      14.306   
4494     S4R2K0             528  M       203    98.79      0.508       1.302   
4495     S4R2K0             528  M       215    98.03      3.940       3.358   
4496     S4R2K0             528  M       226    93.01     -4.021      -2.734   

      x_coord_cb  x_coord_n  y_coord_c  ...  nAA_7.5_180_pae_smooth10  \
0        -16.898    -15.637    -29.957  ...                  1.909091   
1        -19.503    -17.880    -11.797  ...                  1.944444   
2        -20.768    -20.882     -2.522  ...                  2.476190   
3        -26.170    -24.382      3.164  ...                  3.857143   
4        -10.128    -11.907      1.475  ...                  7.619048   
...          ...        ...        ...  ...                       ...   
4492      -3.011     -5.229    -11.159  ...                  7.523810   
4493      13.644     15.422     -7.821  ...                  6.380952   
4494       2.674      1.468      3.585  ...                  7.619048   
4495       2.382      2.708     12.422  ...                  5.761905   
4496      -1.530     -2.851      2.349  ...                  3.866667   

      nAA_8_180_pae_smooth10  nAA_12_180_pae_smooth10  \
0                   1.909091                 3.727273   
1                   1.944444                 4.000000   
2                   2.476190                 5.000000   
3                   3.904762                 7.666667   
4                   8.142857                19.047619   
...                      ...                      ...   
4492                8.571429                25.428571   
4493                7.666667                18.952381   
4494                8.809524                28.714286   
4495                6.714286                18.619048   
4496                4.333333                10.133333   

      nAA_18_180_pae_smooth10  nAA_24_180_pae_smooth10  \
0                    5.545455                 8.454545   
1                    6.555556                10.500000   
2                    9.142857                17.666667   
3                   17.238095                32.380952   
4                   51.666667               104.095238   
...                       ...                      ...   
4492                66.619048               119.285714   
4493                49.142857                89.476190   
4494                82.714286               134.619048   
4495                46.714286                87.380952   
4496                20.333333                38.200000   

      nAA_12_70_pae_smooth10  IDR UniProt UniProt-Primary  \
0                   0.000000    1  A2ADF7          A2ADF7   
1                   0.111111    1  A2ADF7          A2ADF7   
2                   0.285714    1  A2ADF7          A2ADF7   
3                   1.047619    1  A2ADF7          A2ADF7   
4                   5.095238    0  A2ADF7          A2ADF7   
...                      ...  ...     ...             ...   
4492                7.761905    0  S4R2K0          S4R2K0   
4493                4.380952    0  S4R2K0          S4R2K0   
4494                8.571429    0  S4R2K0          S4R2K0   
4495                4.428571    0  S4R2K0          S4R2K0   
4496                1.666667    0  S4R2K0          S4R2K0   

                                      Complete Seque

In [111]:
#mitocarta3_0_methionines_wa.to_csv(os.path.join(curr_dir_path, "Mitochondria_mitocarta_with_alphafold.csv"))

In [112]:
path = os.path.join(curr_dir_path, "Mitochondria_mitocarta_with_alphafold.csv")
mitocarta3_0_methionines_wa = pd.read_csv(path)
mitocarta3_0_methionines_wa.set_index("Unnamed: 0", inplace=True)
mitocarta3_0_methionines_wa.index.name = None
mitocarta3_0_methionines_wa

protein_id  protein_number AA  position  quality  x_coord_c  x_coord_ca  \
0        A2ADF7               1  M         0    34.90    -16.170     -15.807   
1        A2ADF7               1  M         7    25.00    -17.959     -18.801   
2        A2ADF7               1  M        11    31.62    -22.014     -20.790   
3        A2ADF7               1  M        16    60.19    -24.992     -24.872   
4        A2ADF7               1  M        96    86.39    -12.515     -11.617   
...         ...             ... ..       ...      ...        ...         ...   
4492     S4R2K0             528  M        90    98.40     -3.270      -3.856   
4493     S4R2K0             528  M       132    98.22     14.774      14.306   
4494     S4R2K0             528  M       203    98.79      0.508       1.302   
4495     S4R2K0             528  M       215    98.03      3.940       3.358   
4496     S4R2K0             528  M       226    93.01     -4.021      -2.734   

      x_coord_cb  x_coord_n  y_coord_c  ...  nAA_7.5_180_pae_smooth10  \
0        -16.898    -15.637    -29.957  ...                  1.909091   
1        -19.503    -17.880    -11.797  ...                  1.944444   
2        -20.768    -20.882     -2.522  ...                  2.476190   
3        -26.170    -24.382      3.164  ...                  3.857143   
4        -10.128    -11.907      1.475  ...                  7.619048   
...          ...        ...        ...  ...                       ...   
4492      -3.011     -5.229    -11.159  ...                  7.523810   
4493      13.644     15.422     -7.821  ...                  6.380952   
4494       2.674      1.468      3.585  ...                  7.619048   
4495       2.382      2.708     12.422  ...                  5.761905   
4496      -1.530     -2.851      2.349  ...                  3.866667   

      nAA_8_180_pae_smooth10  nAA_12_180_pae_smooth10  \
0                   1.909091                 3.727273   
1                   1.944444                 4.000000   
2                   2.476190                 5.000000   
3                   3.904762                 7.666667   
4                   8.142857                19.047619   
...                      ...                      ...   
4492                8.571429                25.428571   
4493                7.666667                18.952381   
4494                8.809524                28.714286   
4495                6.714286                18.619048   
4496                4.333333                10.133333   

      nAA_18_180_pae_smooth10  nAA_24_180_pae_smooth10  \
0                    5.545455                 8.454545   
1                    6.555556                10.500000   
2                    9.142857                17.666667   
3                   17.238095                32.380952   
4                   51.666667               104.095238   
...                       ...                      ...   
4492                66.619048               119.285714   
4493                49.142857                89.476190   
4494                82.714286               134.619048   
4495                46.714286                87.380952   
4496                20.333333                38.200000   

      nAA_12_70_pae_smooth10  IDR UniProt UniProt-Primary  \
0                   0.000000    1  A2ADF7          A2ADF7   
1                   0.111111    1  A2ADF7          A2ADF7   
2                   0.285714    1  A2ADF7          A2ADF7   
3                   1.047619    1  A2ADF7          A2ADF7   
4                   5.095238    0  A2ADF7          A2ADF7   
...                      ...  ...     ...             ...   
4492                7.761905    0  S4R2K0          S4R2K0   
4493                4.380952    0  S4R2K0          S4R2K0   
4494                8.571429    0  S4R2K0          S4R2K0   
4495                4.428571    0  S4R2K0          S4R2K0   
4496                1.666667    0  S4R2K0          S4R2K0   

                                      Complete Seque

# End